## 講習1 --- IRAFを使ってみる

IRAFのCLコマンドラインを使っての対話的なデータ処理を、 
python + Jupyter notebookからも行うことができます。

まずは、IRAFのタスクを利用して、基本的な画像処理を行ってみます。  
CCDの生データをフラットで割るという処理です。   
IRAFの基本的なタスク、display, imexam, imarith, imcombineを使ってみます。  

サンプルデータとして、すばるSuprime Camで取得したM15(球状星団)のデータを使います。  (SMOKAで公開されているデータです。)  同じディレクトリの ./M15/set1/ および set2/ の中のFITSファイルです。(bzip2で圧縮しているのでbunzip2で展開しておいてください。)

### pyrafのための準備

pythonからIRAFを使うにはpyrafを利用します。  
pyrafを(便利に)使うためには、  

- ホームディレクトリにirafというディレクトリを作成
- そのディレクトリ内で mkirafを実行し、login.cl を作成
- 必要に応じてそのlogin.clを編集  (今回のサンプルデータの場合、set stdimage = imt4096 あるいは imt8192)

をしておきます。 こうしておくと、login.clでの設定がpyraf利用時に反映されます。また、~/iraf/uparm/に各タスク(imexam, imstatなど)のパラメータが保存されます。　　

### モジュールの読み込み

In [1]:
from pyraf import iraf 

これでirafのタスクをpythonで関数として使うことができます。('Warning : sscanf library not ... 'がでることがあります。とりえあず無視してください。)

### Unixコマンド

Unixコマンドを使い、カレントディレクトリおよびその中身を確認することができます。

In [2]:
pwd

'/Users/nakajima/git/wrk/temporal'

In [3]:
ls

M15/                       course1_IRAF_1.ipynb
append_median.ipynb        course2_numpy.ipynb
appendix.ipynb             course3_pyfits.ipynb
appendix1_aplpy.ipynb      course4_matplotlib.ipynb
checkM15.ipynb             course5_IRAF_apphot.ipynb
course0_module.ipynb       img/


M15のディレクトリの中にサンプルのFITSファイルがあります。IRAFのコマンド、display, imexam, imstatを使って、それらファイルの表示やデータの吟味、統計量の測定をしてみましょう。

In [4]:
ls M15/set1

SUPA00320845.fits  SUPA00322525.fits


Supreme Camは10枚のCCDから成ります。
ここでは、まずは5番フレームをサンプルとして使います。  
ファイル名(拡張子除く)の末尾の数字が5のものが、5番フレームです。

<img src='./img/SUPA0032084X.png', width='500'> 

**[notebook tip]** markdownのコメント中にhtmlのタグを使って画像ファイルを、上のように、埋め込むことができます。  
githubでは残念ながら画像埋め込みはレンダリングされないようです。  


### DS9にFITSデータを表示してみる

別のターミナルからds9を立ち上げておきます。

In [5]:
iraf.display('./M15/set1/SUPA00320845.fits', 1)   #  M15  S-Cam 5番フレーム　生データ　I-band 

z1=13102.76 z2=15071.21


In [6]:
iraf.display('./M15/set1/SUPA00322525.fits', 2)   #  ドームフラット 生データ I-band 

z1=20915.8 z2=24668.95


表示されましたね。  
irafモジュールのdisplay()関数を使い、引数としてファイルパス、ds9のフレーム番号、などを指定します。

### imexamを使ってみる
ds9にFITS画像を表示した状態で、次のコマンドを実行し、マウスカーソルを、星のないところにあてて**m**、あるいは星にあてて**a**をタイプしてみましょう。終わるときには**q**をタイプしてください。

In [7]:
iraf.imexam()

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
[1194:1198,2116:2120]       25   14076.   14075.    59.28   13969.   14236.
[1312:1316,2207:2211]       25   14046.   14053.    53.52   13917.   14161.
#   COL    LINE     COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
1278.60 2114.09 1278.60 2114.09
   8.65  11.55 240154.  14085.  18698. 0.21  -77 6.96     2.92     2.97   2.88


imexamでは、**r**とか**e**コマンドを使うとさらに別のグラフィックウィンドウが現れます。ブラウザとかの後ろに隠れているかもしれません。

In [8]:
iraf.imexam('./M15/set1/SUPA00322525.fits', 2 ) 

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
 [895:899,2815:2819]       25   22638.   22631.     153.   22401.   23078.
[1207:1211,2761:2765]       25   22619.   22602.    134.5   22376.   22934.
[1047:1051,2667:2671]       25   22669.   22686.    144.7   22352.   22978.


### imstat を使ってみる
FITS画像の統計量を調べるタスクimstatを使ってみましょう。

In [9]:
iraf.imstat.unlearn()  #  パラメータをデフォルト値に

In [10]:
iraf.imstat('./M15/set1/SUPA00322525.fits')

#               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
 ./M15/set1/SUPA00322525.fits   8528000    22556.     1674.     9595.    43823.


### eparでパラメータ設定 
**iraf.epar('タスク名')** で、IRAFでおなじみのeparのパラメータ設定の画面が別ウィンドウで開きます。GUIでパラメータ設定ができます。同じGUIのウインドウでHELPも読めます。   
(2017-05 MacOS Sierra + Python3.5.3 + Jupyter Notebook では iraf.epar('display')でUnicodeDecodeErrorが生じる)

In [11]:
iraf.epar('imstat')    #    fieldsでmidpt(メジアン)を指定 , lower=0.0


Task imstatistics is running...

#     MIDPT
     22732.


### 変数としてパラメータ設定

Cellでタスクの変数にパラメータを代入してやります。この方法だと、使ったパラメータがこのノートブックに残るので、あとになって「この処理でどんなパラメータ使ったっけ？」となったときに助けになります。

In [14]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 0
iraf.imstat.upper = 30000

In [15]:
iraf.imstat('./M15/set1/SUPA00322525.fits')

#     MIDPT      MEAN    STDDEV
     22689.    22536.     1611.


どんな値がパラメータに入っているかを確認するには、print()してやればよいです。

In [16]:
print (iraf.imstat.fields)

midpt, mean, stddev


In [17]:
iraf.imstat.images = './M15/set1/SUPA00322525.fits'

必要なパラメータが設定されていれば、下のようにタスク名だけで動きます。

In [18]:
iraf.imstat()

#     MIDPT      MEAN    STDDEV
     22689.    22536.     1611.


上ではimstat()の結果が標準出力に表示されました。それぞれの値を変数に保存するには次のようにします。

In [19]:
out = iraf.imstat(format='no', Stdout=1) # format='no' でヘッダ行非表示、Stdout=1で戻り値を返す
v = out[0].split()  #  戻り値はリスト
median = float(v[0])   # 文字列をfloatに変換しておく
mean = float(v[1])
stddev = float(v[2])
print (median)
print (mean)
print ('{0:.1f}'.format(stddev))   #  フォーマット文を使ってみた

22689.35
22536.47
1610.7


デフォルトではStdout=0なので標準出力に値が返ります。

### helpドキュメント

このノートブック内でhelpを読むこともできます。  
helpの表示が縦に長すぎる場合、左の余白部分(In[]: の下あたり)をクリックするとスクロールバーつきウインドウ表示になります。
(私の環境でブラウザChromeの場合にはデフォルトでスクロールバー表示されます。)

In [1]:
#iraf.help('imstat')   #  Githubでは表示が長くなってしまうのでコメントアウトしておきます

### ドームフラットでM15のフレームをフラット処理する

'./M15/set1/SUPA00320845.fits' はM15を観測したIバンドの生データです。  (5番フレーム)  
これを**フラット**で割ることで、CCDの感度ムラと光学系の透過率のムラを補正します。  
視野全体で一様な強度で光っている(と考えている)ものを観測してフラットを作成します。   

_(最近の)CCDはダークやバイアスイメージを生データから引くことはしません。生データをフラットで割るだけです。  (地上観測の近赤外アレイでは、ダークを引いてフラットで割り、スカイバイアスを引くという処理が必要になります。)_   

まず、この**フラット**を作成する必要があります。  
ドームフラットの生データからバイアス値を引いて**規格化**してフラットを作成します。  
もし、CCDの感度ムラがなく、光学系の透過率も完全に一様であれば、フラットは全てのピクセルで1.0の値をもちます。  
通常は複数のフラットの平均から、S/Nのより高いフラットを作成します。

下ではまず、ドームフラット1枚だけからフラット作成します。**imstat**と**imarith**を使用します。  
次に、ドームフラット5枚からフラット作成します。**imcombine**も使用します。  

#### ドームフラット1枚だけを使う

'./M15/set1/SUPA00322525.fits' はドームフラットのIバンドの生データです。  
CCDのフレームには、観測した光に加えて、一様な値がゲタとして加算されています。  
そのゲタの値をオーバースキャン領域から推定します。Suprime Camの5番フレームでは2049列目あたりから右側がオーバースキャン領域です。  

In [21]:
iraf.imstat('./M15/set1/SUPA00322525.fits[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9988.     9981.     55.67


ドームフラットの光があたっている部分のメジアンを求めておきます。

In [22]:
iraf.imstat('./M15/set1/SUPA00322525.fits[1:2048, *]')

#     MIDPT      MEAN    STDDEV
     22769.    22733.      358.


**imarith**の出番です。ゲタの値を引いてから、メジアン値で規格化してフラットを作成しましょう。

In [23]:
iraf.imarith('./M15/set1/SUPA00322525.fits', '-' , 9988, 'flat1.fits')
iraf.imarith('flat1.fits', '/' , 12781, 'flat1.fits')  #  22769 - 9988

このフラットでM15の生データを割ります。このときも、まず、生データからゲタの値を引きます。

In [24]:
iraf.imstat('./M15/set1/SUPA00320845.fits[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9995.     9991.      25.8


In [25]:
iraf.imarith('./M15/set1/SUPA00320845.fits', '-', 9995, 'm15f5.fits')
iraf.imarith('m15f5.fits', '/', 'flat1.fits', 'm15f5.fits')

これでできました。 ds9で表示して確かめてみましょう。  

####  ドームフラット5枚を使う  

'./M15/set2/SUPA003225[0-4]5.fits' はIバンドのドームフラットです。  

In [26]:
import glob # pythonの組み込みモジュール。ワイルドカードを使ったファイル処理など。

In [27]:
flist = glob.glob('./M15/set2/SUPA003225[0-4]5.fits')

これで対象とするファイルのリストを作成します。

In [28]:
print (flist)

['./M15/set2/SUPA00322505.fits', './M15/set2/SUPA00322515.fits', './M15/set2/SUPA00322525.fits', './M15/set2/SUPA00322535.fits', './M15/set2/SUPA00322545.fits']


In [29]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 0
iraf.imstat.upper = 30000

forループを使って、このリストからファイルを一つづつimstatに入力します。

In [30]:
for img in flist:
    iraf.imstat(img + '[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9987.     9981.     56.31
#     MIDPT      MEAN    STDDEV
      9989.     9981.     55.89
#     MIDPT      MEAN    STDDEV
      9988.     9981.     55.67
#     MIDPT      MEAN    STDDEV
      9985.     9980.     55.97
#     MIDPT      MEAN    STDDEV
      9986.     9980.     55.95


上では、glob.globで抽出したリストをいったん変数に代入しましたが、以下のように直接forループに入れても大丈夫です。

In [31]:
iraf.imstat.fields = 'midpt'  #  どうせメジアンしか使わない

for img in glob.glob('./M15/set2/SUPA003225[0-4]5.fits'):
    out1 = iraf.imstat(img + '[2049:2080, *]', format='no', Stdout=1)
    out2 = iraf.imstat(img + '[1:2048, *]', format='no', Stdout=1)
    print (out1, out2)

['9987.463'] ['22980.84']
['9989.1'] ['22851.6']
['9988.168'] ['22768.57']
['9985.175'] ['22850.58']
['9985.746'] ['22848.87']


それでは、各ドームフラットからフラットを作成し、それらをメジアンでコンバインします。 

In [32]:
iraf.imstat.fields = 'midpt'  

num = 0
comstr = ''
for img in glob.glob('./M15/set2/SUPA003225[0-4]5.fits'):
    
    out1 = iraf.imstat(img + '[2049:2080, *]', format='no', Stdout=1)  #  オーバースキャン領域
    out2 = iraf.imstat(img + '[1:2048, *]', format='no', Stdout=1)  #  光のあたってる領域
    med1 = float(out1[0])  #  文字列を数値に変換
    med2 = float(out2[0])
    
    nflat =  'tmp' + str(num) + '.fits'   #  それぞれのフラットを作成
    iraf.imarith(img, '-', med1, nflat)  #  ゲタをひく
    iraf.imarith(nflat, '/', med2 - med1, nflat) #  ゲタを考慮して規格化
    
    num += 1
    comstr += nflat + ','  #  imcombineの引数として与えるための文字列

print (comstr)  #  なぜ下でcomstr[:-1]と、末尾の一文字を削除するか 
    
iraf.imcombine(comstr[:-1], 'iflat5.fits', combine='median')
iraf.imdelete(comstr[:-1])  #  中間ファイルを削除。お掃除お掃除。 


tmp0.fits,tmp1.fits,tmp2.fits,tmp3.fits,tmp4.fits,

Jun 30 15:51: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
              tmp0.fits
              tmp1.fits
              tmp2.fits
              tmp3.fits
              tmp4.fits

  Output image = iflat5.fits, ncombine = 5


これでドームフラットを5枚使ったフラットができました。  
生データをこれで処理してやります。  

In [33]:
iraf.imarith('./M15/set1/SUPA00320845.fits', '-', 9995, 'm15f5n2.fits')
iraf.imarith('m15f5n2.fits', '/', 'iflat5.fits', 'm15f5n2.fits') 

### 練習問題 
4番フレームについて、上と同様の処理を行ってみましょう。  
(1) 生データ'./M15/set1/SUPA00320844.fits' 4番フレームのIバンドの生データをフラットで割る  
1-1 './M15/set2/SUPA00322524.fits' を規格化したものをフラットとして使う。  
1-2 './M15/set2/SUPA003225[0-4]4.fits' から平均のフラットを作成して、それをフラットとして使用する。  

注意 : 4番フレームはオーバースキャン領域が5番とは異なる。  


(2) 